## Imports

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import geopandas as gpd
from shapely.geometry import box
from rasterio.mask import mask
from scipy.spatial import cKDTree
# k-dimensional tree (KDTree) data structure, optimized for efficient nearest-neighbor searches 
# in multi-dimensional space.

In [3]:
solar_file = "data/solar/ottawa_nsrdb_2020.csv"
landcover_file = "data/land/landcover-2020-classification.tif"
dem_file = "data/land/output_be.tif"
dem_slope_file = "data/land/viz/viz.be_slope.tif"
dem_aspect_file = "data/land/viz/viz.be_aspect.tif"
# No protected areas in Ottawa.
grid_lines_file = "data/Ottawa_grid_lines/ottawa_grid_lines.shp"
grid_substations_file = "data/Ottawa_grid_substations/ottawa_substations.shp"

## Slope
There is a .tif file for the slope values in Ottawa region. 

## Land Cover
The suitable lands are chosen. 

In [4]:
bbox = box(-76.5, 44.9, -75.0, 45.6) # min of lon & lat, max of lon & lat
geo = gpd.GeoDataFrame({"geometry":[bbox]}, crs="EPSG:4326")
with rasterio.open(landcover_file) as src:
    lc = src.read(1)
    profile = src.profile
    out_img, out_transform = mask(src, geo.geometry, crop=True)
classes = np.unique(lc)

In [6]:
print(classes)

[ 0  1  2  3  5  6  8 10 11 12 13 14 15 16 17 18 19]


Natural Resources Canada land cover legends https://www150.statcan.gc.ca/n1/pub/16-510-x/16-510-x2025003-eng.htm
Cropland: 2, Sparsely vegetated land: 9, Barren land: 10

In [ ]:
suitable_classes = [2, 9, 10]
lc_mask = np.isin(lc, suitable_classes).astype(np.uint8)

profile.update(dtype=rasterio.uint8)

with rasterio.open("data/processed/ottawa_landcover_mask.tif", "w", **profile) as dst:
    dst.write(lc_mask, 1)

plt.imshow(lc_mask, cmap="Greens")
plt.title("Land Cover Suitability Mask")
plt.show()


: 

## No procted areas

In [ ]:
grid_lines = gpd.read_file(grid_lines_file).to_crs(epsg=3347)

# Sample raster grid
with rasterio.open(dem_fp) as src:
    transform = src.transform
    width, height = src.width, src.height
    xs, ys = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, ys, xs)
    points = np.column_stack([np.ravel(xs), np.ravel(ys)])
    out_meta = src.meta.copy()

# Extract line coordinates
line_coords = np.vstack([
    np.array(geom.coords) 
    for geom in grid_lines.geometry if geom.geom_type == "LineString"
])

# KD-tree nearest distance
tree = cKDTree(line_coords)
distances, _ = tree.query(points, k=1)

distance_raster = distances.reshape(height, width)

out_meta.update(dtype=rasterio.float32)

with rasterio.open("data/processed/ottawa_grid_distance.tif", "w", **out_meta) as dst:
    dst.write(distance_raster.astype(np.float32), 1)

plt.imshow(distance_raster, cmap="plasma")
plt.colorbar(label="Distance to Grid (m)")
plt.title("Transmission Grid Proximity")
plt.show()

In [ ]:
# Load slope mask: allow only slopes < 5°
slope_arr = slope < 5

# Land cover mask
with rasterio.open("data/processed/ottawa_landcover_mask.tif") as src:
    lc_mask = src.read(1).astype(bool)

# Protected mask (invert: 1=protected → unsuitable, but here all 0)
with rasterio.open("data/processed/ottawa_protected_mask.tif") as src:
    protected_mask = src.read(1).astype(bool)

# Final mask
final_mask = slope_arr & lc_mask & (~protected_mask)

plt.imshow(final_mask, cmap="gray")
plt.title("Final Suitability Mask")
plt.show()

# Save
with rasterio.open("data/processed/ottawa_final_suitability.tif", "w", **profile) as dst:
    dst.write(final_mask.astype(rasterio.uint8), 1)
